In [1]:
import sys, os
from tqdm import tqdm
sys.path.append(R"D:\GITHUB\FunAudioLLM-APP\cosyvoice")
sys.path.append(R"D:\GITHUB\FunAudioLLM-APP\cosyvoice\third_party\Matcha-TTS")

from cosyvoice.cli.cosyvoice import CosyVoice, CosyVoice2
from cosyvoice.utils.file_utils import load_wav
from cosyvoice.utils.common import set_all_random_seed
import torchaudio

2025-02-23 13:49:38,448 - modelscope - INFO - PyTorch version 2.3.1+cu121 Found.
2025-02-23 13:49:38,451 - modelscope - INFO - Loading ast index from C:\Users\CHENHAOYUAN\.cache\modelscope\ast_indexer
2025-02-23 13:49:38,669 - modelscope - INFO - Loading done! Current index file version is 1.15.0, with md5 553a05bea23a95025342c8774cbd7108 and a total number of 980 components indexed


failed to import ttsfrd, use WeTextProcessing instead


## 加载模型

In [2]:
cosyvoice = CosyVoice2(R'D:\GITHUB\FunAudioLLM-APP\cosyvoice\pretrained_models\CosyVoice2-0.5B', load_jit=False, load_trt=False, fp16=False)

d:\anaconda3\envs\cosyvoice\lib\site-packages\diffusers\models\lora.py:393: FutureWarning: `LoRACompatibleLinear` is deprecated and will be removed in version 1.0.0. Use of `LoRACompatibleLinear` is deprecated. Please switch to PEFT backend by installing PEFT: `pip install peft`.
  deprecate("LoRACompatibleLinear", "1.0.0", deprecation_message)
2025-02-23 13:49:58,354 INFO input frame rate=25
d:\anaconda3\envs\cosyvoice\lib\site-packages\onnxruntime\capi\onnxruntime_inference_collection.py:69: UserWarning: Specified provider 'CUDAExecutionProvider' is not in available provider names.Available providers: 'AzureExecutionProvider, CPUExecutionProvider'
  warnings.warn(
2025-02-23 13:50:00,398 WETEXT INFO building fst for zh_normalizer ...
2025-02-23 13:50:00,398 INFO building fst for zh_normalizer ...
2025-02-23 13:50:24,821 WETEXT INFO done
2025-02-23 13:50:24,821 INFO done
2025-02-23 13:50:24,822 WETEXT INFO fst path: d:\anaconda3\envs\cosyvoice\lib\site-packages\tn\zh_tn_tagger.fst
202

## 加载音频
需要手动标记一下音频的内容

In [3]:
prompt_audio = load_wav(R'D:\GITHUB\OpenVoice\inputs\train\tvboy1_denoised.wav', 16000)
prompt_text = "两个或者三个吧，嗯肚子很胀，就是，有点，嗯，有时候会吐的"

## 配置

In [16]:
batch = 4 # doesn't work at this moment
output_dir = "./assets/raw_dataset/tvboy_denoised_mandarin_letter_seed34751218"
seed = 34751218

# 以下选一个

## 普通话
input_txts = "./assets/text/withLetter.txt"
cross_lang = ""
skip_line = 0

## 粤语
# input_txts = "./cantonese.txt"
# cross_lang = "粤语"

# ！！下面的不要动

if not os.path.exists(output_dir):
	os.makedirs(output_dir, exist_ok=True)
with open(input_txts, 'r', encoding='utf-8') as f:
	lines = f.readlines()
	input_text_lines = []
	for i in range(0, len(lines), skip_line+1):
		line = lines[i].strip()
		if line == "": continue
		input_text_lines.append(line)
	print(input_text_lines[:5])
	print("Got dataset: ", len(input_text_lines))
	def input_generator():
		for i in range(0, len(input_text_lines), batch):
			yield input_text_lines[i:min(i+batch, len(input_text_lines))]
set_all_random_seed(seed)
print("Output folder: ", output_dir)
print("Using seed: ", seed)

['从A到B的计划需要C和D的配合，以确保E可以成功执行。', '你从F到G的过程中，H的支持是不可缺少的，尤其是在I阶段。', '在项目进行中，J和K的合作将直接影响C的效率。', '如果你想提高L到M的速度，N的调整将非常重要。', '在这次活动中，O的出现为P带来了更大的机会。']
Got dataset:  21
Output folder:  ./assets/raw_dataset/tvboy_denoised_mandarin_letter_seed34751218
Using seed:  34751218


## 测试推理

In [10]:
sample_text = "从A到B需要C来维持，然而若D到E然后从F到G就会让H达不到I隔壁的J。" #K、L、M、N、O、P、Q、R、S、T、U、V、W、X、Y、Z。" # input_text_lines[0]
def zero_infer():
	print("Using zero infer.")
	ret = list(cosyvoice.inference_zero_shot(sample_text, prompt_text, prompt_audio, stream=False))[0]
	torchaudio.save("zero_infer.wav", ret['tts_speech'], cosyvoice.sample_rate)
def cross_infer():
	print("Using cross shot to " + cross_lang + ".")
	instruct_text = f"用{cross_lang}说这句话"
	ret = list(cosyvoice.inference_instruct2(sample_text, instruct_text, prompt_audio, stream=False))[0]
	torchaudio.save("cross_infer.wav", ret['tts_speech'], cosyvoice.sample_rate)
if cross_lang == "":
	zero_infer()
else:
	cross_infer()
print("Done.")

Using zero infer.


  0%|          | 0/1 [00:00<?, ?it/s]

min value is  tensor(-1.0223)
max value is  tensor(1.0314)


2025-02-23 16:18:17,891 INFO synthesis text 从A到B需要C来维持，然而若D到E然后从F到G就会让H达不到I隔壁的J。
2025-02-23 16:18:26,125 INFO yield speech len 11.8, rtf 0.6978050935066352
100%|██████████| 1/1 [00:08<00:00,  8.86s/it]

Done.


## 生成

In [19]:
SkipExisted = False
def zero_shot():
	print("Using zero shot.")
	for idx, input_data in enumerate(tqdm(input_generator())):
		for subidx, data in enumerate(input_data):
			global_idx = idx * batch + subidx + 1
			filename = os.path.join(output_dir, f"{global_idx}.wav")
			if SkipExisted and os.path.exists(filename):
				print("Skip ", filename)
				continue
			ret = list(cosyvoice.inference_zero_shot(data, prompt_text, prompt_audio, stream=False))[0]
			torchaudio.save(filename, ret['tts_speech'], cosyvoice.sample_rate)
def cross_shot():
	print("Using cross shot to " + cross_lang + ".")
	instruct_text = f"用{cross_lang}说这句话"
	for idx, input_data in enumerate(tqdm(input_generator())):
		for subidx, data in enumerate(input_data):
			global_idx = idx * batch + subidx + 1
			filename = os.path.join(output_dir, f"{global_idx}.wav")
			if SkipExisted and os.path.exists(filename):
				print("Skip ", filename)
				continue
			ret = list(cosyvoice.inference_instruct2(data, instruct_text, prompt_audio, stream=False))[0]
			torchaudio.save(filename, ret['tts_speech'], cosyvoice.sample_rate)
if cross_lang == "":
	zero_shot()
else:
	cross_shot()
print("Done.")

Using zero shot.


0it [00:00, ?it/s]

min value is  tensor(-1.0223)
max value is  tensor(1.0314)


2025-02-23 17:57:28,285 INFO synthesis text 从A到B的计划需要C和D的配合，以确保E可以成功执行。
2025-02-23 17:57:33,769 INFO yield speech len 7.08, rtf 0.7744277937937591
100%|██████████| 1/1 [00:06<00:00,  6.08s/it]


min value is  tensor(-1.0223)
max value is  tensor(1.0314)


...

min value is  tensor(-1.0223)
max value is  tensor(1.0314)


2025-02-23 17:59:23,226 INFO synthesis text 你需要从O到Y的每一步都做到精准，以确保C的达成。
2025-02-23 17:59:28,107 INFO yield speech len 6.64, rtf 0.7351493260946619
100%|██████████| 1/1 [00:05<00:00,  5.47s/it]
5it [02:00, 24.13s/it]

min value is  tensor(-1.0223)
max value is  tensor(1.0314)


2025-02-23 17:59:28,705 INFO synthesis text 从A到B需要C来维持，然而若D到E然后从F到G就会让H达不到I隔壁的J。
2025-02-23 17:59:35,343 INFO yield speech len 9.32, rtf 0.7122266446060377
100%|██████████| 1/1 [00:07<00:00,  7.22s/it]
6it [02:07, 21.28s/it]

Done.
